### Day 16: Ticket Translation

As you're walking to yet another connecting flight, you realize that one of the legs of your re-routed trip coming up is on a high-speed train. However, the train ticket you were given is in a language you don't understand. You should probably figure out what it says before you get to the train station after the next flight.

Unfortunately, you can't actually read the words on the ticket. You can, however, read the numbers, and so you figure out the fields these tickets must have and the valid ranges for values in those fields.

You collect the rules for ticket fields, the numbers on your ticket, and the numbers on other nearby tickets for the same train service (via the airport security cameras) together into a single document you can reference (your puzzle input).

The rules for ticket fields specify a list of fields that exist somewhere on the ticket and the valid ranges of values for each field. For example, a rule like class: 1-3 or 5-7 means that one of the fields in every ticket is named class and can be any value in the ranges 1-3 or 5-7 (inclusive, such that 3 and 5 are both valid in this field, but 4 is not).

Each ticket is represented by a single line of comma-separated values. The values are the numbers on the ticket in the order they appear; every ticket has the same format. For example, consider this ticket:

    .--------------------------------------------------------.
    | ????: 101    ?????: 102   ??????????: 103     ???: 104 |
    |                                                        |
    | ??: 301  ??: 302             ???????: 303      ??????? |
    | ??: 401  ??: 402           ???? ????: 403    ????????? |
    '--------------------------------------------------------'
    
Here, ? represents text in a language you don't understand. This ticket might be represented as 101,102,103,104,301,302,303,401,402,403; of course, the actual train tickets you're looking at are much more complicated. In any case, you've extracted just the numbers in such a way that the first number is always the same specific field, the second number is always a different specific field, and so on - you just don't know what each position actually means!

Start by determining which tickets are completely invalid; these are tickets that contain values which aren't valid for any field. Ignore your ticket for now.

For example, suppose you have the following notes:

    class: 1-3 or 5-7
    row: 6-11 or 33-44
    seat: 13-40 or 45-50

    your ticket:
    7,1,14

    nearby tickets:
    7,3,47
    40,4,50
    55,2,20
    38,6,12
    
It doesn't matter which position corresponds to which field; you can identify invalid nearby tickets by considering only whether tickets contain values that are not valid for any field. In this example, the values on the first nearby ticket are all valid for at least one field. This is not true of the other three nearby tickets: the values 4, 55, and 12 are are not valid for any field. Adding together all of the invalid values produces your ticket scanning error rate: 4 + 55 + 12 = 71.

Consider the validity of the nearby tickets you scanned. What is your ticket scanning error rate?


In [1]:
from collections import defaultdict
import re

In [2]:
with open('data/advent_16.txt', 'r') as f:
    data = f.read()

In [3]:
rules, my_ticket, nearby_tickets = data.split('\n\n')

pattern = r'([\w\s]+): (\d+)-(\d+) or (\d+)-(\d+)'

rules = [re.match(pattern, rule).groups() for rule in rules.split('\n')]
rules = [(name, int(a), int(b), int(c), int(d)) for name, a, b, c, d in rules]

my_ticket = my_ticket.split('\n')[1].split(',')

nearby_tickets = [list(map(int, ticket.split(','))) for ticket in nearby_tickets.rstrip('\n').split('\n')[1:]]

In [4]:
error_rate = 0
valid_tickets = []
for ticket in nearby_tickets:
    all_valid = True
    for value in ticket:
        valid = False
        i = 0
        while not valid and i < len(rules):
            _, min_a, max_a, min_b, max_b = rules[i]
            valid = min_a <= value <= max_a 
            valid |= min_b <= value <= max_b
            i += 1
        
        if not valid:
            all_valid = False
            error_rate += value
    
    if all_valid:
        valid_tickets.append(ticket)

In [5]:
print(f"Solution: The ticket scanning error rate is {error_rate}")

Solution: The ticket scanning error rate is 27898



<br> 

### Part Two

Now that you've identified which tickets contain invalid values, discard those tickets entirely. Use the remaining valid tickets to determine which field is which.

Using the valid ranges for each field, determine what order the fields appear on the tickets. The order is consistent between all tickets: if seat is the third field, it is the third field on every ticket, including your ticket.

For example, suppose you have the following notes:

    class: 0-1 or 4-19
    row: 0-5 or 8-19
    seat: 0-13 or 16-19

    your ticket:
    11,12,13

    nearby tickets:
    3,9,18
    15,1,5
    5,14,9
    
Based on the nearby tickets in the above example, the first position must be row, the second position must be class, and the third position must be seat; you can conclude that in your ticket, class is 12, row is 11, and seat is 13.

Once you work out which field is which, look for the six fields on your ticket that start with the word departure. What do you get if you multiply those six values together?

In [6]:
fields_candidates = defaultdict(lambda: [])
for name, min_a, max_a, min_b, max_b in rules:
    
    for position in range(len(rules)):    
        valid = True
        i = 0
        while valid and i < len(valid_tickets):
            value = valid_tickets[i][position]            
            valid &= (min_a <= value <= max_a) or (min_b <= value <= max_b)
            i += 1
    
        if valid:
            fields_candidates[position].append(name)

In [7]:
fields_positions = {}
while len(fields_positions) != len(rules):
    for position, candidates in fields_candidates.items():
        for found in fields_positions.values():
            if found in candidates:
                candidates.remove(found)
  
        if len(candidates) == 1:
            fields_positions[position] = candidates.pop()

In [8]:
res = 1
for idx, field_name in fields_positions.items():
    if field_name.startswith('departure'):
        res *= int(my_ticket[idx])

In [9]:
print(f"Solution: The multiplication of the fields that start with departure is {res}.")

Solution: The multiplication of the fields that start with departure is 2766491048287.
